In [1]:
import torch
from MVCNN.models import MVCNN
from PFT import PruningFineTuner
import numpy as np
from copy import deepcopy
import os
from torchvision import models
import numpy as np

In [2]:
def get_size(model):
    param_size = sum(
        param.nelement() * param.element_size() for param in model.parameters()
    )
    buffer_size = sum(
        buffer.nelement() * buffer.element_size() for buffer in model.buffers()
    )
    return (param_size + buffer_size) / 1024**2

In [ ]:
model = MVCNN.SVCNN('SVCNN')
model.load_state_dict(torch.load('model-00030.pth', map_location='mps'))

print("net_1 size: ", get_size(model.net_1))
print("net_2 size: ", get_size(model.net_2))

net_1 size:  35.17333984375
net_2 size:  456.54700088500977


In [9]:
def count_parameters(model):
    """
    Count the total number of parameters in the model with detailed breakdown.
    
    Args:
        model: PyTorch model
        
    Returns:
        total_params: Total number of parameters
        trainable_params: Number of trainable parameters
        non_trainable_params: Number of frozen parameters
        size_mb: Model size in MB
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    # Calculate model size in MB
    size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    size_mb = size_bytes / (1024 * 1024)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {non_trainable_params:,}")
    print(f"Model size: {size_mb:.2f} MB")
    
    # Optional: Show breakdown by layer
    print("\nParameter breakdown by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")
        
    return total_params, trainable_params, non_trainable_params, size_mb

In [14]:
count_parameters(model.net_1)

Total parameters: 9,220,480
Trainable parameters: 9,220,480
Non-trainable parameters: 0
Model size: 35.17 MB

Parameter breakdown by layer:
0.weight: 1,728 parameters
0.bias: 64 parameters
3.weight: 73,728 parameters
3.bias: 128 parameters
6.weight: 294,912 parameters
6.bias: 256 parameters
8.weight: 589,824 parameters
8.bias: 256 parameters
11.weight: 1,179,648 parameters
11.bias: 512 parameters
13.weight: 2,359,296 parameters
13.bias: 512 parameters
16.weight: 2,359,296 parameters
16.bias: 512 parameters
18.weight: 2,359,296 parameters
18.bias: 512 parameters


(9220480, 9220480, 0, 35.17333984375)

In [30]:
count_parameters(model.net_2)

Total parameters: 119,681,057
Trainable parameters: 119,681,057
Non-trainable parameters: 0
Model size: 456.55 MB

Parameter breakdown by layer:
0.weight: 102,760,448 parameters
0.bias: 4,096 parameters
3.weight: 16,777,216 parameters
3.bias: 4,096 parameters
6.weight: 135,168 parameters
6.bias: 33 parameters


(119681057, 119681057, 0, 456.54700088500977)

In [29]:
count_parameters(model.net_2[0])

Total parameters: 102,764,544
Trainable parameters: 102,764,544
Non-trainable parameters: 0
Model size: 392.02 MB

Parameter breakdown by layer:
weight: 102,760,448 parameters
bias: 4,096 parameters


(102764544, 102764544, 0, 392.015625)

In [3]:
def get_model():
    model = MVCNN.SVCNN('SVCNN')
    model.load_state_dict(torch.load('mvcnn.pth', map_location='mps'))
    model = deepcopy(model)
        
    print(f"Model size: {get_size(model):.2f} MB")
    
    # x, y = model.net_1, model.net_2
    # del model.net_1
    # del model.net_2
    
    # model.features = x
    # model.classifier = y
    
    return model

In [4]:
def write_to_file(pruning_amount, size):
    if 'results_again.csv' not in os.listdir():
        with open('results_again.csv', 'w') as f:
            f.write("Pruning Amount, Model Size\n")
    with open('results_again.csv', 'a') as f:
        f.write(f"{pruning_amount}, {size}\n")

In [5]:
def get_vgg_model():
    model = models.vgg11(weights=models.VGG11_Weights.IMAGENET1K_V1)
    x, y, z = model.avgpool, model.classifier, model.features

    model.avgpool = None
    model.classifier = None
    model.features = None

    del model.classifier
    del model.features
    del model.avgpool

    model.net_1 = z
    model.avgpool = x
    model.net_2 = y
    model.net_2[6] = torch.nn.Linear(4096, 33)
    
    return model

In [6]:
# pruner = PruningFineTuner(get_vgg_model())
# pruning_amount = 48

# model = pruner.prune(pruning_amount)
# print(f"Model size after pruning: {get_size(model):.2f} MB")

In [7]:
get_model()

Model size: 491.72 MB


SVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [8]:
# Create your original pruning amounts
pruning_amounts = np.arange(0, 100, 4)

# Center value
center = np.mean(pruning_amounts)

# Compute Gaussian-based probabilities
sigma = 15  # controls how "tight" around center you prefer; adjust if needed
probabilities = np.exp(-0.5 * ((pruning_amounts - center) / sigma) ** 2)
probabilities /= probabilities.sum()  # normalize to sum to 1

# Sample without replacement according to the probabilities
shuffled_pruning_amounts = np.random.choice(pruning_amounts, size=len(pruning_amounts), replace=False, p=probabilities)

# Now shuffled_pruning_amounts starts more around the middle
pruning_amounts = shuffled_pruning_amounts

In [9]:
pruning_amounts

array([36, 28, 44, 48, 76, 52, 16, 80, 20, 56, 40, 64, 72, 32, 68, 24, 60,
       88, 84,  4, 12, 96,  8, 92,  0])

In [10]:
print(f"Pruning amounts: {pruning_amounts}")
for pruning_amount in pruning_amounts:
    pruner = PruningFineTuner(get_model())
    model = pruner.prune(pruning_amount, True)
    print(f"Model size after {pruning_amount}% pruning: {get_size(model):.2f} MB")
    write_to_file(pruning_amount, get_size(model))
    pruner.reset()
    del model
    del pruner

Pruning amounts: [36 28 44 48 76 52 16 80 20 56 40 64 72 32 68 24 60 88 84  4 12 96  8 92
  0]
Model size: 491.72 MB
Total Filters to prune: 990 For Pruning Percentage: 36
Layers that will be pruned {0: 17, 6: 78, 16: 178, 13: 209, 18: 190, 11: 204, 8: 96, 3: 18}
Pruning Filters
Total Filters Pruned: 990
Model size after 36% pruning: 324.98 MB
PruningFineTuner object deleted and memory cleared.
Model size: 491.72 MB
Total Filters to prune: 770 For Pruning Percentage: 28
Layers that will be pruned {16: 153, 6: 79, 18: 158, 11: 149, 8: 69, 13: 138, 0: 11, 3: 13}
Pruning Filters
Total Filters Pruned: 770
Model size after 28% pruning: 353.60 MB
PruningFineTuner object deleted and memory cleared.
Model size: 491.72 MB
Total Filters to prune: 1210 For Pruning Percentage: 44
Layers that will be pruned {0: 24, 11: 263, 18: 221, 8: 112, 6: 110, 16: 213, 13: 239, 3: 28}
Pruning Filters


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108533a10>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


: 

: 

In [22]:
def _get_pruning_amounts(start, end, step):
        pruning_amounts = np.arange(start, end, step)
        center = np.mean(pruning_amounts)

        sigma = 15  # controls how "tight" around center you prefer; adjust if needed
        probabilities = np.exp(-0.5 * ((pruning_amounts - center) / sigma) ** 2)
        probabilities /= probabilities.sum()  # normalize to sum to 1

        pruning_amounts = np.random.choice(
            pruning_amounts,
            size=len(pruning_amounts),
            replace=False,
            p=probabilities,
        )

        done = [80, 43, 58, 41, 66, 17, 61, 60, 27, 34, 50, 26, 29, 81, 52]
        print(f"Done pruning amounts: {len(done)}")

        # Filter out the done pruning amounts
        pruning_amounts = [amount for amount in pruning_amounts if amount not in done]
        print(f"Remaining pruning amounts: {len(pruning_amounts)}")
        
        print(f"Percentage Done: {len(done) / len(pruning_amounts) * 100:.2f}%")
        
        return pruning_amounts

In [24]:
_get_pruning_amounts(0, 100, 1)

Done pruning amounts: 15
Remaining pruning amounts: 85
Percentage Done: 17.65%


[np.int64(44),
 np.int64(51),
 np.int64(38),
 np.int64(45),
 np.int64(69),
 np.int64(40),
 np.int64(39),
 np.int64(64),
 np.int64(49),
 np.int64(54),
 np.int64(56),
 np.int64(48),
 np.int64(46),
 np.int64(32),
 np.int64(35),
 np.int64(57),
 np.int64(36),
 np.int64(76),
 np.int64(28),
 np.int64(70),
 np.int64(24),
 np.int64(37),
 np.int64(93),
 np.int64(53),
 np.int64(65),
 np.int64(47),
 np.int64(83),
 np.int64(67),
 np.int64(55),
 np.int64(62),
 np.int64(73),
 np.int64(23),
 np.int64(25),
 np.int64(42),
 np.int64(22),
 np.int64(63),
 np.int64(33),
 np.int64(68),
 np.int64(14),
 np.int64(59),
 np.int64(72),
 np.int64(15),
 np.int64(71),
 np.int64(31),
 np.int64(74),
 np.int64(30),
 np.int64(75),
 np.int64(90),
 np.int64(79),
 np.int64(19),
 np.int64(12),
 np.int64(8),
 np.int64(21),
 np.int64(18),
 np.int64(11),
 np.int64(77),
 np.int64(78),
 np.int64(16),
 np.int64(87),
 np.int64(13),
 np.int64(86),
 np.int64(82),
 np.int64(84),
 np.int64(20),
 np.int64(92),
 np.int64(89),
 np.int64(8